In [17]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [18]:
for filename in os.listdir("../data/RawData/BLS"):
    print(filename)

banana_1980_2021.csv
beef_1984_2021.csv
bread_1980_2021.csv
chicken_1980_2021.csv
coffee_1980_2021.csv
eggs_1980_2021.csv
electricity_1980_2021.csv
flour_1980_2021.csv
rice_1980_2021.csv
unleadedGasoline_1980_2021.csv


In [19]:
def toQuarter(month):
    if month>=1 and month<=3:
        return 1
    elif month > 3 and month <=6:
        return 2
    elif month > 6 and month <= 9:
        return 3
    else: 
        return 4

In [20]:
def formatData(filename):
    data = pd.read_csv('../data/RawData/BLS/'+filename,usecols=["Year","Period","Value"])
    
    data.Period = data.Period.apply(lambda x: x[1:]).astype("int")
    data["Quarter"] = data.Period.apply(toQuarter)
    data = data.drop(columns=["Period"])
    data = data.groupby(["Year","Quarter"]).mean()
    return data

In [89]:
def combine(df,left_on_col = ["YEAR FORECAST MADE","QUARTER"]):
    returnFrame = df.copy()
    for filename in os.listdir("../data/RawData/BLS"):
        to_merge = formatData(filename).reset_index(level=[0,1])
        to_merge.rename(columns={"Value":filename.split("_")[0]},inplace=True)
        returnFrame = returnFrame.merge(to_merge,how="left",left_on=left_on_col,right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
        #returnFrame = returnFrame.merge(to_merge,how="left",left_on=left_on_col,right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
    returnFrame = returnFrame[returnFrame.columns.drop(returnFrame.filter(regex='drop').columns)]
    return returnFrame.drop(columns=["Year","Quarter"])
    

In [87]:
training = pd.read_csv("../data/TrainingData/fulldata.csv")
training.head()

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,bin_value_sum,TDIST,actual,ACTUAL_BIN,MAX,NUM_OF_MAX,ACTUAL_CONF,HIT,pred_average,pred_var
0,1968,4,1968,NominalGNP,1,NaN,1968-1-1968-4,NominalGNP-1968-1-1968-4,0.0,0.0,...,100.0,0.0,9.40449,2.0,0.6,1.0,0.00,0.0,6.00,0.4500
1,1968,4,1968,NominalGNP,2,NaN,1968-2-1968-4,NominalGNP-1968-2-1968-4,0.0,0.0,...,100.0,0.0,9.40449,2.0,0.4,1.0,0.00,0.0,6.10,1.0400
2,1968,4,1968,NominalGNP,3,NaN,1968-3-1968-4,NominalGNP-1968-3-1968-4,0.0,0.0,...,100.0,0.0,9.40449,2.0,0.5,1.0,0.00,0.0,6.80,0.4100
3,1968,4,1968,NominalGNP,4,NaN,1968-4-1968-4,NominalGNP-1968-4-1968-4,0.0,0.0,...,100.0,0.0,9.40449,2.0,0.4,1.0,0.00,0.0,6.15,1.3275
4,1968,4,1968,NominalGNP,5,NaN,1968-5-1968-4,NominalGNP-1968-5-1968-4,3.0,4.0,...,100.0,0.0,9.40449,2.0,0.3,1.0,0.04,0.0,4.65,7.2275


In [94]:
combine(training).to_csv("../data/TrainingData/trainingWithItems.csv")

In [95]:
quarterFred= pd.read_csv("../data/CleanData/QuarterlyFred.csv",index_col=0)
quarterFred["DATE"] = quarterFred["DATE"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
quarterFred["Year"] = quarterFred["DATE"].apply(lambda x:x.year)
quarterFred["Month"] = quarterFred["DATE"].apply(lambda y:y.month)
quarterFred["Quarter"] = quarterFred["Month"].apply(toQuarter)
quarterFred.drop(columns=["Month"],inplace=True)
quarterFred.head()

,DATE,GDPC1_PCH,GNP_PCH,GNPC96_PCH,CPILFESL,PCEPILFE,JHDUSRGDPBR,UNRATE,Year,Quarter
0,1947-04-01,-0.26669,1.19643,-0.23978,NaN,NaN,NaN,NaN,1947,2
1,1947-07-01,-0.20650,1.47857,-0.20822,NaN,NaN,NaN,NaN,1947,3
2,1947-10-01,1.56421,4.09427,1.58636,NaN,NaN,NaN,NaN,1947,4
3,1948-01-01,1.50426,2.35726,1.54854,NaN,NaN,NaN,3.733333,1948,1
4,1948-04-01,1.65066,2.58785,1.67265,NaN,NaN,NaN,3.666667,1948,2


In [96]:
combine(quarterFred,["Year","Quarter"]).to_csv("../data/CleanData/QuarterlyFredWithItems.csv",index=False)